In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import numpy as np
import pandas as pd
import pdb
import sklearn
from sklearn.metrics import mean_squared_error as mse

### Import Datasets

In [12]:
data_path = '/content/drive/MyDrive/STATS307/Data/'
cases_df = pd.read_csv(data_path + 'covid19_confirmed_US.csv')
cases_df = pd.DataFrame(cases_df.iloc[:, 11:].diff(axis=1).sum(axis=0)).reset_index()\
              .rename(columns={'index':'date', 0:'cases'})

# string_df = pd.read_csv(data_path + 'strindex.csv')
# indices_df = pd.read_csv(data_path + 'indices20-22.csv')
visitors_df = pd.read_csv(data_path + 'changeinvisitors.csv')

In [13]:
cases_df.head(3)

,date,cases
0,1/22/20,0.0
1,1/23/20,0.0
2,1/24/20,1.0


In [14]:
# string_df.head(3)

In [15]:
visitors_df.head(3)

,Entity,Code,Day,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,United States,USA,2020-02-17,6.333,1.0,1.0,-1.333,19.667,-7.333
1,United States,USA,2020-02-18,4.750,0.5,1.0,-0.750,16.250,-5.500
2,United States,USA,2020-02-19,4.200,0.4,0.8,-0.400,14.600,-4.200


In [16]:
# indices_df.head(3)

### Categoricals - dont run

In [17]:
# convert to categorical variables
# from pandas.api.types import CategoricalDtype
#cat_type = CategoricalDtype(categories=["0.0", "1.0", "2.0", "3.0"], ordered=True)

C1 has 4 cats 
0 - no measures,
1 - recommend closing or all schools open with  alterations resulting in 
significant differences compared to non-Covid-19 operations, 
2 - require closing (only some levels or categories, eg just high school, 
or just public schools), 
3 - require closing all levels)

C2 has 4 categoriess too
0 - no measures
1 - recommend closing (or recommend work from home) or all businesses open with alterations resulting in significant differences compared to non-Covid-19 operation
2 - require closing (or work from home) for some sectors or categories of workers
3 - require closing (or work from home) for all-but-essential workplaces (eg grocery stores, doctors)

C4 has 4 categories
0 - no restrictions
1 - restrictions on very large gatherings (the limit is above 1000 people)
2 - restrictions on gatherings between 101-1000 people
3 - restrictions on gatherings between 11-100 people
4 - restrictions on gatherings of 10 people or less

C5 has 3 categories:
0 - no measures
1 - recommend closing (or significantly reduce volume/route/means of transport available)
2 - require closing (or prohibit most citizens from using it)

C6 has 4:
0 - no measures
1 - recommend not leaving house
2 - require not leaving house with exceptions for daily exercise, grocery shopping, and 'essential' trips
3 - require not leaving house with minimal exceptions (eg allowed to leave once a week, or only one person can leave at a time, etc)

C7 has 3:
0 - no measures
1 - recommend not to travel between regions/cities
2 - internal movement restrictions in place

C8 has 4:
0 - no restrictions
1 - screening arrivals
2 - quarantine arrivals from some or all regions
3 - ban arrivals from some regions
4 - ban on all regions or total border closure

### X

Convert date to Datetime Objects

In [18]:
cases_df['date'] = pd.to_datetime(cases_df['date'])
# string_df['date'] = pd.to_datetime(string_df['date'])
visitors_df['Day'] = pd.to_datetime(visitors_df['Day'])
# indices_df['Date'] = pd.to_datetime(indices_df['Date'], format='%Y%m%d')

In [19]:
# merged = string_df.merge(indices_df, left_on='date', right_on='Date')\
#           .merge(visitors_df, left_on='Date', right_on='Day')
merged = visitors_df

In [20]:
merged.head()

,Entity,Code,Day,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,United States,USA,2020-02-17,6.333,1.000,1.000,-1.333,19.667,-7.333
1,United States,USA,2020-02-18,4.750,0.500,1.000,-0.750,16.250,-5.500
2,United States,USA,2020-02-19,4.200,0.400,0.800,-0.400,14.600,-4.200
3,United States,USA,2020-02-20,3.667,0.333,0.833,-0.333,12.833,-3.500
4,United States,USA,2020-02-21,3.429,0.000,0.714,-0.143,11.571,-3.000


In [21]:
X = merged.loc[~merged.isna().any(axis=1)]
X.drop(labels=['Entity', 'Code'], axis=1, inplace=True) #'location', 'Date', 
X.rename(columns={'Day':'date'},inplace=True)
# X.iloc[:,2], X.iloc[:,1] = X.iloc[:,1], 'United States'
# X.rename(columns={'stringency_index':'CountryName', 'CountryName':'stringency_index'}, inplace=True)

In [22]:
X.head()

,date,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,2020-02-17,6.333,1.000,1.000,-1.333,19.667,-7.333
1,2020-02-18,4.750,0.500,1.000,-0.750,16.250,-5.500
2,2020-02-19,4.200,0.400,0.800,-0.400,14.600,-4.200
3,2020-02-20,3.667,0.333,0.833,-0.333,12.833,-3.500
4,2020-02-21,3.429,0.000,0.714,-0.143,11.571,-3.000


In [23]:
Y = cases_df.merge(X['date'], left_on='date', right_on='date')

In [24]:
Y.head()

,date,cases
0,2020-02-17,1.0
1,2020-02-18,1.0
2,2020-02-19,0.0
3,2020-02-20,0.0
4,2020-02-21,0.0


In [25]:
X.isnull().any()
Y.isnull().any()

date     False
cases    False
dtype: bool

In [ ]:
#X.to_csv('X.csv')
#Y.to_csv('Y.csv')

# Model

In [26]:
!pip install pmdarima
import pmdarima as pm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 7.6 MB/s 
     |████████████████████████████████| 9.9 MB 53.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [27]:
# June 2020 - June 2021
X_train = X[(X['date'] >= '2020-06-01') & (X['date'] < '2021-06-01')].iloc[:,1:]
y_train = Y[(Y['date'] >= '2020-06-01') & (Y['date'] < '2021-06-01')]['cases']

In [28]:
# First 2 weeks of June 2021
X_test = X[(X['date'] < '2021-06-15') & (X['date'] >= '2021-06-01')].iloc[:,1:]
y_test = Y[(Y['date'] < '2021-06-15') & (Y['date'] >= '2021-06-01')]['cases']

Drop columns with identical values

In [29]:
X_train.columns

Index(['retail_and_recreation', 'grocery_and_pharmacy', 'residential',
       'transit_stations', 'parks', 'workplaces'],
      dtype='object')

In [30]:
nonunique_cols = []
for c in range(len(X_train.columns)):
  if len(X_train.iloc[:,c].unique())==1:
    nonunique_cols.append(c)
nonunique_cols

[]

In [31]:
X_train = X_train.drop(X_train.columns[nonunique_cols], axis = 1)
X_test = X_test.drop(X_test.columns[nonunique_cols], axis = 1)

Normalize numerical variables

In [32]:
norm_cols = ['retail_and_recreation', 'grocery_and_pharmacy', 'residential', 'transit_stations', 'parks', 'workplaces'] #'stringency_index', 
for c in norm_cols:
  X_train[c] = (X_train[c]-X_train[c].mean())/X_train[c].std()
  X_test[c] = (X_test[c]-X_test[c].mean())/X_test[c].std()
X_train.head(5)

,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
105,-0.988311,0.423363,1.335020,-0.536295,0.545753,-0.560379
106,-0.891537,0.517148,1.268997,-0.511770,0.580223,-0.494323
107,-0.770528,0.673236,1.202973,-0.462719,0.585152,-0.461295
108,-0.697821,0.766801,1.136950,-0.413840,0.654093,-0.428498
109,-0.625114,0.860585,1.070927,-0.364789,0.678705,-0.395470


In [33]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((365, 6), (365,), (14, 6), (14,))

Stationarity + GC Test

In [34]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import kpss

In [35]:
df_train = X_train.copy(deep=True)
df_train.dtypes
df_y = y_train.copy(deep=True)

### ADF test: Null is non-stationary

In [36]:
def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

In [38]:
# retail + recreation check
print('ADF Test: retail time series')
adf_test(df_train['retail_and_recreation'])
# grocery and pharmacy
print('ADF Test: grocery time series')
adf_test(df_train['grocery_and_pharmacy'])
# residential 
print('ADF Test: residential time series')
adf_test(df_train['residential'])
# transit_stations
print('ADF Test: transit_stations time series')
adf_test(df_train['transit_stations'])
# parks
print('ADF Test: parks time series')
adf_test(df_train['parks'])
# workplaces 
print('ADF Test: workplaces time series')
adf_test(df_train['workplaces'])
# y check
print('ADF Test: y time series')
adf_test(df_y)

ADF Test: retail time series
ADF Statistics: -0.803531
p-value: 0.818083
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: grocery time series
ADF Statistics: -1.016907
p-value: 0.747048
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: residential time series
ADF Statistics: -1.595743
p-value: 0.485732
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: transit_stations time series
ADF Statistics: -1.231293
p-value: 0.659991
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: parks time series
ADF Statistics: -1.120774
p-value: 0.706717
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: workplaces time series
ADF Statistics: -3.393754
p-value: 0.011174
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: y time series
ADF Statistics: -1.433400
p-value: 0.566163
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571


All above seem to be non-stationary as p-values are > 0.05 except workplace so will run KPSS to cross-check.

### KPSS TEST: Null is stationary

In [39]:
def kpss_test(df):    
    statistic, p_value, n_lags, critical_values = kpss(df.values)
    print(f'KPSS Statistic: {statistic}', f'p-value: {p_value}', f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')

In [40]:

# retail + recreation check
print('KPSS Test: retail time series')
kpss_test(df_train['retail_and_recreation'])
# grocery and pharmacy
print('KPSS Test: grocery time series')
kpss_test(df_train['grocery_and_pharmacy'])
# residential 
print('KPSS Test: residential time series')
kpss_test(df_train['residential'])
# transit_stations
print('KPSS Test: transit_stations time series')
kpss_test(df_train['transit_stations'])
# parks
print('KPSS Test: parks time series')
kpss_test(df_train['parks'])
# workplaces 
print('KPSS Test: workplaces time series')
kpss_test(df_train['workplaces'])

KPSS Test: retail time series
KPSS Statistic: 0.6345915604829887 p-value: 0.0194916763197283 num lags: 11
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: grocery time series
KPSS Statistic: 0.5938567081449306 p-value: 0.02319484471409721 num lags: 11
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: residential time series
KPSS Statistic: 0.5352245563670627 p-value: 0.033733208025436334 num lags: 11
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: transit_stations time series
KPSS Statistic: 0.5530868227289282 p-value: 0.029710175061052207 num lags: 11
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: parks time series
KPSS Statistic: 1.5271459292731697 p-value: 0.01 num lags: 11
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: workplaces time series
KPSS Statistic: 0.6984408072818054 p-value: 0.013687199

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


KPSS suggests all are non-stationary so will difference/attempt to make non-stationary

Stationarity of Xt

In [41]:
df_train = df_train.loc[:, ~df_train.columns.isin(['C1M_School.closing', 'C2M_Workplace.closing',
                                                   'C6M_Stay.at.home.requirements'])]

In [42]:
df_y = df_y.diff().dropna()

In [43]:
df_train.dtypes
df_train = df_train.diff().dropna()

In [45]:
# retail + recreation check
print('ADF Test: retail time series')
adf_test(df_train['retail_and_recreation'])
# grocery and pharmacy
print('ADF Test: grocery time series')
adf_test(df_train['grocery_and_pharmacy'])
# residential 
print('ADF Test: residential time series')
adf_test(df_train['residential'])
# transit_stations
print('ADF Test: transit_stations time series')
adf_test(df_train['transit_stations'])
# parks
print('ADF Test: parks time series')
adf_test(df_train['parks'])
# workplaces 
print('ADF Test: workplaces time series')
adf_test(df_train['workplaces'])
# y check
print('ADF Test: y time series')
adf_test(df_y)

ADF Test: retail time series
ADF Statistics: -4.905421
p-value: 0.000034
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: grocery time series
ADF Statistics: -6.095875
p-value: 0.000000
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: residential time series
ADF Statistics: -4.912715
p-value: 0.000033
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: transit_stations time series
ADF Statistics: -3.223105
p-value: 0.018686
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: parks time series
ADF Statistics: -3.448443
p-value: 0.009414
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: workplaces time series
ADF Statistics: -5.268502
p-value: 0.000006
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
ADF Test: y time series
ADF Statistics: -3.915958
p-value: 0.001920
Critical values:
	1%: -3.449
	5%: -2.870
	10%: -2.571


In [46]:
df_train_trans = df_train.copy(deep=True)
df_train_trans = pd.concat([df_train_trans, df_y], axis=1, ignore_index=False)
df_train_trans.head()

,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,cases
106,0.096773,0.093784,-0.066023,0.024525,0.034470,0.066056,2207.0
107,0.121009,0.156088,-0.066023,0.049051,0.004929,0.033028,-975.0
108,0.072707,0.093566,-0.066023,0.048879,0.068941,0.032797,8722.0
109,0.072707,0.093784,-0.066023,0.049051,0.024612,0.033028,-1469.0
110,0.072538,0.062523,-0.066023,0.024525,0.014788,-0.033028,-6184.0


All p-values are below 0.05 and so, first differencing is enough for converting xt and yt into stationary time series.

### GRANGER CAUSALITY

In [48]:
# GRANGER TEST ASSUMES X AND Y ARE STATIONARY SO FIRST DIFF MADE THEM STATIONARY
from statsmodels.tsa.stattools import grangercausalitytests

maxlag=15
test1 = 'params_ftest'
test2 = 'ssr_ftest' 

def grangers_causation_matrix(data, variables, test, verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

# params f test
grangers_causation_matrix(df_train_trans, variables = df_train_trans.columns, test=test1, verbose=False)

,retail_and_recreation_x,grocery_and_pharmacy_x,residential_x,transit_stations_x,parks_x,workplaces_x,cases_x
retail_and_recreation_y,0.0000,0.0000,0.0000,0.0000,0.4628,0.0000,0.0000
grocery_and_pharmacy_y,0.0000,0.0000,0.0000,0.0000,0.4340,0.0000,0.0000
residential_y,0.0000,0.0000,0.0000,0.0003,0.2906,0.0000,0.0001
transit_stations_y,0.0000,0.0000,0.0374,0.0000,0.1975,0.0000,0.0000
parks_y,0.5791,0.5695,0.0117,0.1900,0.0000,0.0788,0.6923
workplaces_y,0.0000,0.0000,0.0000,0.0000,0.0295,0.0000,0.0047
cases_y,0.0000,0.0000,0.0000,0.0000,0.3064,0.0000,0.0000


In [49]:
# ssr f test
grangers_causation_matrix(df_train_trans, variables = df_train_trans.columns, test=test2, verbose=False)

,retail_and_recreation_x,grocery_and_pharmacy_x,residential_x,transit_stations_x,parks_x,workplaces_x,cases_x
retail_and_recreation_y,1.0000,0.0000,0.0000,0.0000,0.4628,0.0000,0.0000
grocery_and_pharmacy_y,0.0000,1.0000,0.0000,0.0000,0.4340,0.0000,0.0000
residential_y,0.0000,0.0000,1.0000,0.0003,0.2906,0.0000,0.0001
transit_stations_y,0.0000,0.0000,0.0374,1.0000,0.1975,0.0000,0.0000
parks_y,0.5791,0.5695,0.0117,0.1900,1.0000,0.0788,0.6923
workplaces_y,0.0000,0.0000,0.0000,0.0000,0.0295,1.0000,0.0047
cases_y,0.0000,0.0000,0.0000,0.0000,0.3064,0.0000,1.0000


minimum p value for 15 lags, X causes Y. only tests 2 time series at a time

In [ ]:
# df = df_train_trans
# reduction1 = df[df.columns[df.columns.isin(['cases', 'retail_and_recreation'])]]
# columns_titles = ['cases', 'parks']
# reduction=reduction1.reindex(columns=columns_titles)
# grangercausalitytests(reduction1, maxlag = maxlag, verbose=True)

### Data Manipulation for X_train, y_train, X_test, y_test for numerical variables

In [ ]:
X_test.dtypes
## first differencing on only numerical variables
df_x_train = df_train
df_x_test = X_test.loc[:, ~X_test.columns.isin(['C1M_School.closing', 'C2M_Workplace.closing', 'C6M_Stay.at.home.requirements'])]
df_x_test = df_x_test
# log on y train
df_y_train = y_train.copy(deep=True)
df_y_train = np.log(df_y_train)
# first differencing on test + log
df_y_test = np.log(y_test)              

In [ ]:
df_x_train.shape, df_y_train.shape

((365, 6), (365,))

## Model 1: ARIMA on only stationary numerical variables/time series

In [ ]:
US_model = {}
USmodel_AIC = {}
UStest_MSE = {}

In [ ]:
arima = pm.auto_arima(df_y_train, X=df_x_train, d= None, seasonal=False, start_p=0, start_q=0, max_order=5, \
                      test='adf', error_action='ignore', suppress_warnings=True, \
                      stepwise=True, trace=True) #len(y_train_ldiff)

US_model['ARIMA'] = arima
USmodel_AIC['ARIMA'] = arima.aic()
arima_ = arima.fit(df_y_train, df_x_train, n_period=len(df_y_train))
y_pred =  arima_.predict(n_periods=len(df_x_test), X=df_x_test, return_conf_int=False, alpha=0.05)
y_true = df_y_test
error = mse(y_pred, y_true)
UStest_MSE['ARIMA'] = error

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-158.059, Time=0.38 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-161.742, Time=0.48 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-164.729, Time=1.19 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-159.937, Time=0.24 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-188.007, Time=1.47 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-194.984, Time=1.46 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-166.712, Time=0.93 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-208.088, Time=1.50 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-177.817, Time=1.12 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=-226.672, Time=1.89 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=-201.751, Time=1.62 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=-250.762, Time=2.09 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=-242.770, Time=1.59 sec
 ARIMA(5,1,2)(0,0,0)[0] intercept   : AIC=-378.363, Time=2.10 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
# Quarterly
arima = pm.auto_arima(df_y_train, X=df_x_train, start_p = 0, d = None, 
                        start_q=0, max_p=5, max_d=2, max_q=5, start_P = 1, D=None,
                        start_Q = 1, m=4, seasonal=True,
                        test='adf', error_action='ignore', suppress_warnings=True, 
                        stepwise=True, trace=True)
US_model['SARIMA-Q'] = arima
USmodel_AIC['SARIMA-Q'] = arima.aic()
arima_ = arima.fit(df_y_train, df_x_train, n_period=len(df_y_train))
y_pred =  arima_.predict(n_periods=len(df_x_test), X=df_x_test, return_conf_int=False, alpha=0.05)
y_true = df_y_test
error = mse(y_pred, y_true)
UStest_MSE['SARIMA-Q'] = error

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[4] intercept   : AIC=-165.090, Time=1.11 sec
 ARIMA(0,1,0)(0,0,0)[4] intercept   : AIC=-158.059, Time=0.29 sec
 ARIMA(1,1,0)(1,0,0)[4] intercept   : AIC=-176.936, Time=1.63 sec
 ARIMA(0,1,1)(0,0,1)[4] intercept   : AIC=-182.652, Time=2.33 sec
 ARIMA(0,1,0)(0,0,0)[4]             : AIC=-159.937, Time=0.66 sec
 ARIMA(0,1,1)(0,0,0)[4] intercept   : AIC=-164.729, Time=1.18 sec
 ARIMA(0,1,1)(1,0,1)[4] intercept   : AIC=-197.587, Time=1.91 sec
 ARIMA(0,1,1)(1,0,0)[4] intercept   : AIC=-191.238, Time=1.55 sec
 ARIMA(0,1,1)(2,0,1)[4] intercept   : AIC=-202.789, Time=2.55 sec
 ARIMA(0,1,1)(2,0,0)[4] intercept   : AIC=-204.777, Time=3.37 sec
 ARIMA(0,1,0)(2,0,0)[4] intercept   : AIC=-164.314, Time=1.93 sec
 ARIMA(1,1,1)(2,0,0)[4] intercept   : AIC=-209.185, Time=2.52 sec
 ARIMA(1,1,1)(1,0,0)[4] intercept   : AIC=-198.470, Time=1.72 sec
 ARIMA(1,1,1)(2,0,1)[4] intercept   : AIC=-207.268, Time=3.29 sec
 ARIMA(1,1,1)(1,0,1)[4] intercept

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
# Monthly
arima = pm.auto_arima(df_y_train, X=df_x_train, start_p = 0, d = None, 
                        start_q=0, max_p=5, max_d=2, max_q=5, start_P = 1, D=None,
                        start_Q = 1, m=12, seasonal=True,
                        test='adf', error_action='ignore', suppress_warnings=True, 
                        stepwise=True, trace=True)
US_model['SARIMA-M'] = arima
USmodel_AIC['SARIMA-M'] = arima.aic()
arima_ = arima.fit(df_y_train, df_x_train, n_period=len(df_y_train))
y_pred =  arima_.predict(n_periods=len(df_x_test), X=df_x_test, return_conf_int=False, alpha=0.05)
y_true = df_y_test
error = mse(y_pred, y_true)
UStest_MSE['SARIMA-M'] = error

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[12] intercept   : AIC=-170.877, Time=2.82 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=-158.059, Time=0.29 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=-172.721, Time=2.62 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=-183.285, Time=3.12 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=-159.937, Time=0.24 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=-164.729, Time=1.16 sec
 ARIMA(0,1,1)(1,0,1)[12] intercept   : AIC=-183.935, Time=3.33 sec
 ARIMA(0,1,1)(1,0,0)[12] intercept   : AIC=-174.803, Time=2.45 sec
 ARIMA(0,1,1)(2,0,1)[12] intercept   : AIC=-206.615, Time=6.88 sec
 ARIMA(0,1,1)(2,0,0)[12] intercept   : AIC=-207.255, Time=5.70 sec
 ARIMA(0,1,0)(2,0,0)[12] intercept   : AIC=-176.044, Time=4.89 sec
 ARIMA(1,1,1)(2,0,0)[12] intercept   : AIC=-208.477, Time=7.16 sec
 ARIMA(1,1,1)(1,0,0)[12] intercept   : AIC=-191.459, Time=2.95 sec
 ARIMA(1,1,1)(2,0,1)[12] intercept   : AIC=-207.775, Time=7.77 sec
 ARIMA(1,1,1)(1,0,1

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
# Yearly
arima = pm.auto_arima(df_y_train, X=df_x_train, start_p = 0, d = None, 
                        start_q=0, max_p=5, max_d=2, max_q=5, start_P = 1, D=None,
                        start_Q = 1, m=1, seasonal=True,
                        test='adf', error_action='ignore', suppress_warnings=True, 
                        stepwise=True, trace=True)
US_model['SARIMA-A'] = arima
USmodel_AIC['SARIMA-A'] = arima.aic()
arima_ = arima.fit(df_y_train, df_x_train, n_period=len(df_y_train))
y_pred =  arima_.predict(n_periods=len(df_x_test), X=df_x_test, return_conf_int=False, alpha=0.05)
y_true = df_y_test
error = mse(y_pred, y_true)
UStest_MSE['SARIMA-A'] = error

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-158.059, Time=0.30 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-161.742, Time=0.50 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-164.729, Time=1.16 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-159.937, Time=0.23 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-188.007, Time=1.40 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-194.984, Time=1.40 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-166.712, Time=0.93 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-208.088, Time=1.51 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-177.817, Time=1.14 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=-226.672, Time=1.97 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=-201.751, Time=1.63 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=-250.762, Time=2.06 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=-242.770, Time=1.52 sec
 ARIMA(5,1,2)(0,0,0)[0] intercept   : AIC=-378.363, Time=2.07 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
list(y_pred)

[9.458088753878101,
 9.537782382533914,
 9.641969250718859,
 9.80703311526756,
 9.670979023412055,
 9.29576349196867,
 8.846586048431215,
 9.271018422981124,
 9.585086822391508,
 9.81215251592152,
 9.732643375138142,
 9.504667966580822,
 9.170717976403866,
 9.072170995873112]

In [ ]:
UStest_MSE

{'ARIMA': 0.15466564964474566,
 'SARIMA-Q': 0.3920328169368628,
 'SARIMA-M': 0.2914862846692943,
 'SARIMA-A': 0.15466564964474566}

In [ ]:
arima.aic()


-378.7577271130553

In [ ]:
map['US'].append(arima)

In [ ]:
map['US']

[ARIMA(order=(5, 1, 2), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 ARIMA(order=(5, 1, 2), scoring_args={}, suppress_warnings=True,
       with_intercept=False)]

In [ ]:
arima_ = arima.fit(df_y_train, df_x_train, n_period=len(df_y_train))

In [ ]:
y_pred = arima_.predict(n_periods=len(df_x_test), X=df_x_test, return_conf_int=False, alpha=0.05)

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
y_true = df_y_test
mse(y_pred, y_true)

4.558998193870246

## Model 2: ARIMA with both stationary numerical and non-stationary categorical

In [ ]:
# get categorical variables for one less than total
cat_train, cat_test = X_train, X_test
cat_test.dtypes

In [ ]:
cat_train = cat_train.loc[:, ["C1M_School.closing","C2M_Workplace.closing","C6M_Stay.at.home.requirements"]]
cat_train = cat_train.iloc[1: ,]

cat_test = cat_test.loc[:, ["C1M_School.closing","C2M_Workplace.closing","C6M_Stay.at.home.requirements"]]
cat_test = cat_test.iloc[1: ,]

In [ ]:
full_df_train = pd.concat([df_train, cat_train], axis = 1)
full_df_test = pd.concat([df_x_test, cat_test], axis = 1)

In [ ]:
full_df_test.shape

In [ ]:
arima = pm.auto_arima(df_y_train, m=len(df_y_train), seasonal=False, start_p=0, start_q=0, max_order=4, \
                      test='adf', error_action='ignore', suppress_warnings=True, \
                      stepwise=True, trace=True)

In [ ]:
arima_ = arima.fit(df_y_train, full_df_train, n_period=len(df_y_train))

In [ ]:
y_pred = arima_.predict(n_periods=len(full_df_test), X=full_df_test, return_conf_int=False, alpha=0.05)

In [ ]:
y_true = df_y_test
mse(y_pred, y_true)

## Model 3: SARIMAX: ?-Day Forecast

## Model 4: LSTM 
#### chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://cloud.r-project.org/web/packages/EEMDlstm/EEMDlstm.pdf

Failed Attempt @ Simple AR(2)

In [ ]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

for i in [1,2,3,4,5,6,7,8,9,10]:
    model = VAR(X_train)
    results = model.fit(i)
    print('Order={}, AIC={}, BIC={}'.format(i, results.aic, results.bic))
# Choose AR(2) since 2nd smallest AIC/ smallest BIC

In [ ]:
ar_results = VAR(X_train).fit(2).summary()

In [ ]:
model = VAR(X_train)
model.select_order(15)
results = model.fit(maxlags=15, ic='aic')

In [ ]:
model.predict(y_train, X_train, 400)
lag_order = results.k_ar
results.forecast(y_train.values[-lag_order:], 15)

Attempt @ ARIMA (non-differenced xt)

In [ ]:
y_train_ldiff = np.log(y_train).diff().dropna()
arima = pm.auto_arima(y_train_ldiff, m=365, seasonal=False, start_p=0, start_q=0, max_order=4, \
                      test='adf', error_action='ignore', suppress_warnings=True, \
                      stepwise=True, trace=True) #len(y_train_ldiff)

In [ ]:
arima_ = arima.fit(y_train_ldiff, X_train.iloc[1:,:], n_period=len(X_train)-1)

In [ ]:
y_pred = arima_.predict(n_periods=len(X_test), X=X_test, return_conf_int=False, alpha=0.05)

In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error as mse
y_test_ldiff = np.log(y_test).diff().dropna()
print(len(np.log(y_test).diff()[1:]), len(y_pred))
mse(y_test_ldiff, y_pred[:-1])

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
rcParams['figure.figsize'] = 6, 3
plot_pacf(y_train_ldiff) 
plot_acf(y_train_ldiff)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
Y_ = Y.copy(deep=True)
Y_.set_index('date', inplace=True)
decomposition = sm.tsa.seasonal_decompose(Y_['cases'], model='additive')
fig = decomposition.plot()
plt.show()

In [ ]:
Y.set_index('date')

In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
#plt.plot(y_test)
plt.plot(y_pred)
plt.title('Daily New Confirmed Cases in the US')
plt.ylabel('# Cases')
plt.xlabel('Date')
# plt.xticks(np.arange(1, len(Y['date'])+1, 14))
plt.xticks(rotation=90)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae
np.sqrt(mse(y_test, y_pred))

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
rcParams['figure.figsize'] = 7, 4
plt.plot(Y['date'], Y['cases'])
plt.title('Daily New Confirmed Cases in the US')
plt.ylabel('# Cases')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.show()


In [ ]:
from pmdarima.arima import ADFTest 
adf_test = ADFTest(alpha=0.05).should_diff(Y['cases'])
adf_test

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
plt.figure(figsize=[5, 3])
plot_pacf(Y['cases']) 
plot_acf(Y['cases'])

In [ ]:
ad_fuller_result = adfuller(daily_cases['cases'])
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
data = daily_cases.copy(deep=True)
data.rename(columns={'cases':'data'}, inplace=True)
data['data'] = np.log(data['data']).diff()
data = data.drop(data.index[0])

In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
plt.plot(data['data'])
plt.title("Log Difference of Quarterly EPS for Johnson & Johnson")
plt.show()

In [ ]:
plot_pacf(data['data']);
plot_acf(data['data']);

In [ ]:
ad_fuller_result = adfuller(data['data'].iloc[1:])
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')